In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install chainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967716 sha256=a3af3d68976477d3e727d7705ee0d2813182c5878a170f3e89a5ea65e992c8c8
  Stored in directory: /root/.cache/pip/wheels/c4/95/6a/16014db6f761c4e742755b64aac60dbe142da1df6c5919f790
Successfully built chainer


In [ ]:
import os
import numpy as np
import random


import argparse
import numpy as np

import chainer
import chainer.functions as F
import chainer.links as L

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training,serializers
from chainer.training import extensions
from PIL import Image

In [ ]:
# Define location of the images and labels
xbd_dir="/content/drive/MyDrive/XBD/train/"
images_dir=os.path.join(xbd_dir,"images")
labels_dir=os.path.join(xbd_dir,"labels")
masks_dir = os.path.join(xbd_dir, 'masks')
train_data_info=os.path.join(xbd_dir,"train_data_info.csv")

train_batch_size = 16
test_batch_size = 4
epochs = 100
frequency = 25
log_dir = os.path.join(xbd_dir,"logs")

In [ ]:
# Read mask and convert to array
def read_mask_image(path, dtype):
    with Image.open(path) as img:
        #convert image to binary format
        img = img.convert('1')
        image = np.array(img, dtype=dtype)
    return image

In [ ]:
# Read image and convert to array
def read_disaster_image(path, dtype):
    with Image.open(path) as img:
        image = np.array(img, dtype=dtype)
    return image

In [ ]:
from chainer.dataset import dataset_mixin
class CustomDataset(dataset_mixin.DatasetMixin):
    def __init__(self, train_test_data_path, images_path, masks_path, image_dtype=np.float32, mask_dtype=np.int32, mean=0, crop_size=400, is_test=False):
        with open(train_test_data_path) as file:
            image_mask_pairs = []
            for line in file:
                line = line.rstrip('\n')
                image_filename = label_filename = line
                image_mask_pairs.append((image_filename, label_filename))
        self.image_mask_pairs = image_mask_pairs
        self.images_path = images_path
        self.masks_path = masks_path
        self.image_dtype = image_dtype
        self.mask_dtype = mask_dtype
        self.mean = mean[np.newaxis, np.newaxis, :]
        self.crop_size = crop_size
        self.is_test = is_test

    def __len__(self):
        return len(self.image_mask_pairs)

    def get_example(self, index):
        image_filename, mask_filename = self.image_mask_pairs[index]
        images_path = os.path.join(self.images_path, image_filename)
        image = read_disaster_image(images_path, self.image_dtype)
        image = (image - self.mean) / 255.0
        masks_path = os.path.join(self.masks_path, mask_filename)
        mask_image = read_mask_image(masks_path, self.mask_dtype)
        h, w, c = image.shape
        mask = np.zeros(shape=[h, w], dtype=np.int32)
        mask[mask_image > 0] = 1
        #compare the size of image to desired crop size
        #If image size is less than crop size add padding to ensure size requirement
        if (h < self.crop_size) or (w < self.crop_size):
            H, W = max(h, self.crop_size), max(w, self.crop_size)
            padding_x1, padding_y1 = (W - w) // 2, (H - h) // 2
            padding_x2, padding_y2 = (W - w - padding_x1), (H - h - padding_y1)
            image = np.pad(image, ((padding_y1, padding_y2), (padding_x1, padding_x2), (0, 0)), 'symmetric')
            if self.is_test:
                mask = np.pad(mask, ((padding_y1, padding_y2), (padding_x1, padding_x2)), 'constant', constant_values=255)
            else:
                mask = np.pad(mask, ((padding_y1, padding_y2), (padding_x1, padding_x2)), 'symmetric')
            h, w = H, W
            if not self.is_test:
                # Randomly flip horizontally
                if random.choice([True, False]):
                    image = image[:, ::-1, :]
                    mask = mask[:, ::-1]
                # Randomly flip vertically
                if random.choice([True, False]):
                    image = image[::-1, :, :]
                    mask = mask[::-1, :]
                # Randomly crop
                top = random.randint(0, h - self.crop_size)
                left = random.randint(0, w - self.crop_size)
            else:
                top = (h - self.crop_size) // 2
                left = (w - self.crop_size) // 2
            bottom = top + self.crop_size
            right = left + self.crop_size
            image = image[top:bottom, left:right]
            mask = mask[top:bottom, left:right]
            print(image.transpose(2, 0, 1), mask)
        return image.transpose(2, 0, 1), mask


In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L


class UNet(chainer.Chain):

    def __init__(self):
        super(UNet, self).__init__(
            c0=L.Convolution2D(3, 32, 3, 1, 1),
            c1=L.Convolution2D(32, 64, 4, 2, 1),
            c2=L.Convolution2D(64, 64, 3, 1, 1),
            c3=L.Convolution2D(64, 128, 4, 2, 1),
            c4=L.Convolution2D(128, 128, 3, 1, 1),
            c5=L.Convolution2D(128, 256, 4, 2, 1),
            c6=L.Convolution2D(256, 256, 3, 1, 1),
            c7=L.Convolution2D(256, 512, 4, 2, 1),
            c8=L.Convolution2D(512, 512, 3, 1, 1),

            dc8=L.Deconvolution2D(1024, 512, 4, 2, 1),
            dc7=L.Convolution2D(512, 256, 3, 1, 1),
            dc6=L.Deconvolution2D(512, 256, 4, 2, 1),
            dc5=L.Convolution2D(256, 128, 3, 1, 1),
            dc4=L.Deconvolution2D(256, 128, 4, 2, 1),
            dc3=L.Convolution2D(128, 64, 3, 1, 1),
            dc2=L.Deconvolution2D(128, 64, 4, 2, 1),
            dc1=L.Convolution2D(64, 32, 3, 1, 1),
            dc0=L.Convolution2D(64, 2, 3, 1, 1),
            bnc0=L.BatchNormalization(32),
            bnc1=L.BatchNormalization(64),
            bnc2=L.BatchNormalization(64),
            bnc3=L.BatchNormalization(128),
            bnc4=L.BatchNormalization(128),
            bnc5=L.BatchNormalization(256),
            bnc6=L.BatchNormalization(256),
            bnc7=L.BatchNormalization(512),
            bnc8=L.BatchNormalization(512),
            bnd8=L.BatchNormalization(512),
            bnd7=L.BatchNormalization(256),
            bnd6=L.BatchNormalization(256),
            bnd5=L.BatchNormalization(128),
            bnd4=L.BatchNormalization(128),
            bnd3=L.BatchNormalization(64),
            bnd2=L.BatchNormalization(64),
            bnd1=L.BatchNormalization(32)
        )


    def forward(self, x):
        x = x.astype(np.float32)
        e0 = F.relu(self.bnc0(self.c0(x)))
        e1 = F.relu(self.bnc1(self.c1(e0)))
        e2 = F.relu(self.bnc2(self.c2(e1)))
        del e1
        e3 = F.relu(self.bnc3(self.c3(e2)))
        e4 = F.relu(self.bnc4(self.c4(e3)))
        del e3
        e5 = F.relu(self.bnc5(self.c5(e4)))
        e6 = F.relu(self.bnc6(self.c6(e5)))
        del e5
        e7 = F.relu(self.bnc7(self.c7(e6)))
        e8 = F.relu(self.bnc8(self.c8(e7)))

        d8 = F.relu(self.bnd8(self.dc8(F.concat([e7, e8]))))
        del e7, e8
        d7 = F.relu(self.bnd7(self.dc7(d8)))
        del d8
        d6 = F.relu(self.bnd6(self.dc6(F.concat([e6, d7]))))
        del d7, e6
        d5 = F.relu(self.bnd5(self.dc5(d6)))
        del d6
        d4 = F.relu(self.bnd4(self.dc4(F.concat([e4, d5]))))
        del d5, e4
        d3 = F.relu(self.bnd3(self.dc3(d4)))
        del d4
        d2 = F.relu(self.bnd2(self.dc2(F.concat([e2, d3]))))
        del d3, e2
        d1 = F.relu(self.bnd1(self.dc1(d2)))
        del d2
        d0 = self.dc0(F.concat([e0, d1]))
        return d0


    def __call__(self, x, t):
        h = self.forward(x)
        loss = F.softmax_cross_entropy(h, t, ignore_label=255)
        accuracy = F.accuracy(h, t, ignore_label=255)
        chainer.report({'loss': loss, 'accuracy': accuracy}, self)
        return loss

In [ ]:
# Initialize UNet()
model = UNet()

chainer.cuda.get_device_from_id(0).use()
model.to_gpu()
print(model)

UNet(
  (bnc0): BatchNormalization(size=32, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc1): BatchNormalization(size=64, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc2): BatchNormalization(size=64, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc3): BatchNormalization(size=128, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc4): BatchNormalization(size=128, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc5): BatchNormalization(size=256, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc6): BatchNormalization(size=256, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc7): BatchNormalization(size=512, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc8): BatchNormalization(size=512, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnd1): BatchNormalizat

In [ ]:
# Setup Adam optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [ ]:
# Load Train and Test Datasets
mean = np.load(os.path.join(xbd_dir, "mean.npy"))

train_dataset = CustomDataset(os.path.join(xbd_dir, "train.txt"), images_dir, masks_dir, mean=mean, crop_size=400, is_test=False)

test_dataset = CustomDataset (os.path.join(xbd_dir, "test.txt"), images_dir, masks_dir, mean=mean, crop_size=480, is_test=True)

train_iterator = chainer.iterators.SerialIterator(train_dataset, train_batch_size,shuffle=True)
test_iterator = chainer.iterators.SerialIterator(test_dataset, test_batch_size, shuffle=False)

In [ ]:
# Set up a trainer
updater = training.StandardUpdater(train_iterator, optimizer, device=0)
trainer = training.Trainer(updater, (epochs, 'epoch'), out=log_dir)

In [ ]:
# Evaluate the model with the test dataset
trainer.extend(extensions.Evaluator(test_iterator, model, device=0))

In [ ]:
# logging
trainer.extend(extensions.LogReport())

# Print report for each epoch
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss','main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar
trainer.extend(extensions.ProgressBar())

In [ ]:
# Run the trainer
trainer.run()

GPU: 0
# Minibatch-size: 16
# Crop-size: 400
# epoch: 100



/usr/local/lib/python3.10/dist-packages/chainer/functions/connection/convolution_2d.py:295: PerformanceWarning: The best algo of conv fwd might not be selected due to lack of workspace size (8388608)
  cuda.cudnn.convolution_forward(


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time


/usr/local/lib/python3.10/dist-packages/chainer/functions/connection/convolution_2d.py:295: PerformanceWarning: The best algo of conv fwd might not be selected due to lack of workspace size (8388608)
  cuda.cudnn.convolution_forward(


1           0.264956    0.246204              0.893785       0.932104                  86.4053       
2           0.164318    0.202324              0.94255        0.928895                  170.204       
     total [#.................................................]  2.84%
this epoch [#########################################.........] 83.69%
       100 iter, 2 epoch / 100 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
3           0.154632    0.183874              0.945492       0.929173                  253.75        
4           0.15428     0.194857              0.943795       0.927305                  337.22        
5           0.149064    0.170035              0.944023       0.933043                  422.436       
     total [##................................................]  5.67%
this epoch [#################################.................] 67.38%
       200 iter, 5 epoch / 100 epochs
   0.41267 iters/sec. Estimated time to finish: 2:14:17.377440.
6        

In [ ]:
model_save_path = '/content/drive/MyDrive/XBD/localization_model'
chainer.serializers.save_npz(model_save_path, model)